# Simulating NLOS captures with mitransient

## Overview
<div class="admonition important alert alert-block alert-success">

🚀 **You will learn how to:**

<ul>
  <li>Import `mitransient` and configure it for NLOS scenes</li>
  <li>Setup a NLOS scene, including the relay wall, laser, camera and hidden geometry</li>
  <li>Visualize the output capture</li>
</ul>

</div>

In this tutorial, you will use mitransient to simulate a non-line-of-sight (NLOS) scene with mitransient. If you are not familiar, NLOS imaging algorithm use indirect light scattered in the scene to reconstruct hidden objects, for example around a corner. For this example, we will target a Z-shaped geometry defined in `Z.obj`. The scene's camera will not directly capture light from the Z object, but instead capture indirect light that has bounced on a rectangular *relay wall*. The key elements in the scene will be:
1. The **hidden Z-shaped object**
2. The **relay wall**, defined with the `rectangle` plugin, which contains a `nlos_capture_meter` (a **sensor** to measure indirect light)
3. A **laser source** that illuminates the relay wall. We will show how to position and orient this source so that it illuinates points on the relay wall

### Importing `mitransient`
Before importing `mitransient`, you need to import Mitsuba 3 and set a variant (here we use `llvm_ad_rgb`). Only if you want to use more variants, you will need to compile Mitsuba 3 (not `mitransient`) yourself. And you will need to add the compilation folder to the `PYTHONPATH` (see the commented code for how to do that).

<div class="admonition important alert alert-block alert-warning">

For many cases, `mitransient` requires the use of a `llvm_*` or `cuda_*` variant, so we don't recommend using `scalar_rgb`. It will work with any `llvm_*` or `cuda_*` variant, and for most use cases `llvm_ad_rgb` or `cuda_ad_rgb` is enough. **Maybe if you plan to do NLOS simulations it can be worth to do `llvm_mono` or `cuda_mono`, which only uses one wavelength instead of three RGB channels, and thus is faster.**

</div>

In [ ]:
# If you have compiled Mitsuba 3 yourself, you will need to specify the path
# to the compilation folder
# import sys
# sys.path.insert(0, '<mitsuba-path>/mitsuba3/build/python')
import mitsuba as mi
# To set a variant, you need to have set it in the mitsuba.conf file
# https://mitsuba.readthedocs.io/en/latest/src/key_topics/variants.html
mi.set_variant('cuda_ad_mono_polarized')

import mitransient as mitr

### Setup the NLOS scene

In this tutorial you will learn how to prepare a NLOS scene using our library. We will do it following a programmatically way defining each component independently. Here we do it with `mi.load_dict`, but we also provide a `nlos_Z.xml` file which can be loaded with `mi.load_file` and gives the same results. Note that the conversion from XML and dictionaries is fairly straightforward.

<div class="admonition important alert alert-block alert-success">

If you want to learn more about the parameters of each plugin (geometry, emitter, transient_film, etc.), you can check [Mitsuba 3's documentation](https://mitsuba.readthedocs.io/en/stable/src/plugin_reference.html), and our documentation ([integrators](https://mitransient.readthedocs.io/en/latest/generated/plugin_reference/section_integrators.html), [films](https://mitransient.readthedocs.io/en/latest/generated/plugin_reference/section_films.html), [sensors](https://mitransient.readthedocs.io/en/latest/generated/plugin_reference/section_sensors.html), [other functions](https://mitransient.readthedocs.io/en/latest/src/other.html))

</div>

In [ ]:
# Load the geometry of the hidden scene
gold_bsdf = {
    "type": "roughconductor",
    "distribution": "ggx",
    "material": "Au",
    "alpha_u": 0.9,
    "alpha_v": 0.9
}

geometry = mi.load_dict(
    {
        "type": "obj",
        "filename": "./Z.obj",
        "to_world": mi.ScalarTransform4f().translate([0.0, 0.0, 1.0]),
        "bsdf": gold_bsdf,
    }
)

# Load the emitter (laser) of the scene
emitter = mi.load_dict(
    {
        "type": "projector",
        "irradiance": 100.0,
        "fov": 0.2,
        "to_world": mi.ScalarTransform4f().translate([0.0, 0.0, 0.25]),
    }
)

# Define the transient film which store all the data
transient_film = mi.load_dict(
    {
        "type": "transient_hdr_film",
        "width": 64,
        "height": 64,
        "temporal_bins": 300,
        "bin_width_opl": 0.006,
        "start_opl": 1.85,
        "rfilter": {"type": "box"},
    }
)

# Define the sensor of the scene
nlos_sensor = mi.load_dict(
    {
        "type": "nlos_capture_meter",
        "sampler": {"type": "independent", "sample_count": 65_536},
        "account_first_and_last_bounces": False,
        # This config sets the nlos_sensor in front of the relay wall, not realistic for
        # NLOS setups, but it is easier for polarization visualization
        "sensor_origin": mi.ScalarPoint3f(0.0, 0.0, 0.25),
        "transient_film": transient_film,
    }
)

# Load the relay wall. This includes the custom "nlos_capture_meter" sensor which allows to setup measure points directly on the shape and importance sample paths going through the relay wall.
relay_wall = mi.load_dict(
    {
        "type": "rectangle",
        "bsdf": gold_bsdf,
        "nlos_sensor": nlos_sensor,
    }
)

# Finally load the integrator
integrator = mi.load_dict(
    {
        "type": "transient_nlos_path",
        "nlos_laser_sampling": True,
        "nlos_hidden_geometry_sampling": True,
        "nlos_hidden_geometry_sampling_do_rroulette": False,
        "temporal_filter": "box",
    }
)

In [ ]:
# Assemble the final scene
scene = mi.load_dict({
    'type' : 'scene',
    'geometry' : geometry,
    'emitter' : emitter,
    'relay_wall' : relay_wall,
    'integrator' : integrator
})


In [ ]:
# Now we focus the emitter to irradiate one specific pixel of the "relay wall"
pixel = mi.Point2f(32, 32)
mitr.nlos.focus_emitter_at_relay_wall_pixel(pixel, relay_wall, emitter)

### Render the scene in steady and transient domain

In [ ]:
data_steady, data_transient = mi.render(scene)

<div class="admonition important alert alert-block alert-warning">
Mitsuba 3 and `mitransient` work with Dr.JIT, which has lazy evaluation.
That means the actual image/video will not be computed until you use it.
As such, this cell should take &lt;1s to execute
</div>

The result is:
1) A steady state image `data_steady` with dimensions (width, height, channels)
2) A transient image `data_transient` with dimensions (width, height, time, channels)

`data_steady` would be the result of a conventional (non-transient) render i.e. `data_steady = data_transient.sum(axis=2)`

### Visualize the steady and transient image
We provide different functions so you can visualize your data in a Jupyter notebook

### Visualize the transient image

The important part for NLOS imaging is `data_transient`, which contains the time-resolved indirect illumination. Here we show how to visualize it.

In [ ]:
'''
data_transient is a 4D array represented as a Dr.JIT's TensorXf class.
It is very similar to a numpy array (in fact, you can convert between
the two by using np.array(data_transient))
'''
print(data_transient.__class__.__name__)
# The channels represent (x, y, time, rgb)
print(data_transient.shape)

### Plot radiance at one pixel over time

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

i, j = 11, 11

# There are two main ways of plotting data_transient
# The first one is to plot a single pixel's time-resolved response
plt.plot(np.array(data_transient)[i, j, :, 0])
plt.xlabel('Time index')
plt.ylabel(f'Captured radiance at pixel ({i}, {j})')
plt.show()

## Transient-polarization visualization

S1 and S2 share colorbar.

The other plots are those proposed by Wilkie and Weidlich [2013], commonly used in polarization research (see Baek et al. [2020] teaser)

[Wilkie2013] Alexander Wilkie and Andrea Weidlich. 2010. A standardised polarisation visualisation for images. In Proceedings of the 26th Spring Conference on Computer Graphics (SCCG '10). Association for Computing Machinery, New York, NY, USA, 43–50. https://doi.org/10.1145/1925059.1925070

[Baek2020] Seung-Hwan Baek, Tizian Zeltner, Hyun Jin Ku, Inseung Hwang, Xin Tong, Wenzel Jakob, and Min H. Kim. 2020. Image-based acquisition and modeling of polarimetric reflectance. ACM Trans. Graph. 39, 4, Article 139 (August 2020), 14 pages. https://doi.org/10.1145/3386569.3392387


In [ ]:
data_transient_np = np.array(data_transient)
print(f'{data_transient_np.shape=}')
dop = mitr.vis.degree_of_polarization(data_transient_np)
dop, aolp, aolp_scaled, top, chirality = mitr.vis.polarization_generate_false_color(data_transient_np)

In [ ]:
mitr.vis.show_video_polarized(data_transient_np[:, :, :, :], dop, aolp, top, chirality, save_path='video.mp4', display_method=mitr.vis.DisplayMethod.ShowVideo, show_false_color=True)